In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
X_train = pd.read_csv('input/train.csv', header=None, prefix='col')
X_test = pd.read_csv('input/test.csv', header=None, prefix='col') 
y_train = pd.read_csv('input/train-target.csv', header=None, names=['target']) 
subm = pd.read_csv('input/sample-submission.csv', header=None) 

df = pd.concat([X_train, y_train], axis=1)

# Preproc

In [3]:
drop_cols = ['col17', 'col22', 'col26', 'col9', 'col15', 'col16']
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

In [4]:
mask = [ True, False,  True, False, False,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True, False,  True,
       False, False, False, False,  True,  True]

In [5]:
from sklearn.decomposition import FastICA

ica = FastICA(n_components=9, random_state=12, max_iter=1500)
X_train_ica = ica.fit_transform(X_train.values)
X_test_ica = ica.transform(X_test.values)

In [6]:
# from sklearn.decomposition import TruncatedSVD

# svd = TruncatedSVD(n_components=10, random_state=12)
# X_train_svd = svd.fit_transform(X_train.values[:, mask])
# X_test_svd = svd.transform(X_test.values[:, mask])

In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=11, random_state=13, svd_solver='randomized')
X_train_pca = pca.fit_transform(X_train.values[:, mask])
X_test_pca = pca.transform(X_test.values[:, mask])

In [8]:
from sklearn.decomposition import KernelPCA

pca = KernelPCA(n_components=11, random_state=13, kernel='linear')
X_train_pca = pca.fit_transform(X_train.values[:, mask])
X_test_pca = pca.transform(X_test.values[:, mask])

In [9]:
# X = np.concatenate([X_train.values, X_train_ica], axis=1)
# y = y_train.values.flatten()
# X_test_np = np.concatenate([X_test.values, X_test_ica], axis=1)

X = np.concatenate([X_train.values[:, mask], X_train_ica, X_train_pca], axis=1)
y = y_train.values.flatten()
X_test_np = np.concatenate([X_test.values[:, mask], X_test_ica, X_test_pca], axis=1)

# X = X_train.values
# y = y_train.values.flatten()
# X_test_np = X_test.values

# X = X_train.values[:, mask]
# y = y_train.values.flatten()
# X_test_np = X_test.values[:, mask]

In [16]:
# CV mean score: 0.8388, std: 0.0120.

rf_params = {'n_estimators': 500, 'max_depth': None, 'n_jobs': -1, 
             'min_samples_split': 5, 'random_state': 0}

oof_rf, prediction_rf, scores = train_other_model(X, X_test_np, y, params=rf_params, 
                                                    folds=folds, model_type='rf')

Fold 0 started at Sat Aug 15 13:22:35 2020
Fold 1 started at Sat Aug 15 13:22:39 2020
Fold 2 started at Sat Aug 15 13:22:43 2020
Fold 3 started at Sat Aug 15 13:22:48 2020
Fold 4 started at Sat Aug 15 13:22:52 2020
CV mean score: 0.8388, std: 0.0120.


In [121]:
# from sklearn.decomposition import KernelPCA

# for kernel in ['sigmoid','cosine']:
#     print(kernel)
#     pca = KernelPCA(n_components=11, random_state=13, kernel=kernel)
#     X_train_pca = pca.fit_transform(X_train.values[:, mask])
#     X_test_pca = pca.transform(X_test.values[:, mask])
    
#     X = np.concatenate([X_train.values[:, mask], X_train_ica, X_train_pca], axis=1)
#     y = y_train.values.flatten()
#     X_test_np = np.concatenate([X_test.values[:, mask], X_test_ica, X_test_pca], axis=1)
    
#     oof_rf, prediction_rf, scores = train_other_model(X, X_test_np, y, params=rf_params, 
#                                                     folds=folds, model_type='rf')

sigmoid
Fold 0 started at Thu Aug 13 16:27:11 2020
Fold 1 started at Thu Aug 13 16:27:15 2020
Fold 2 started at Thu Aug 13 16:27:19 2020
Fold 3 started at Thu Aug 13 16:27:22 2020
Fold 4 started at Thu Aug 13 16:27:26 2020
CV mean score: 0.8345, std: 0.0119.
cosine
Fold 0 started at Thu Aug 13 16:28:40 2020
Fold 1 started at Thu Aug 13 16:28:44 2020
Fold 2 started at Thu Aug 13 16:28:48 2020
Fold 3 started at Thu Aug 13 16:28:51 2020
Fold 4 started at Thu Aug 13 16:28:55 2020
CV mean score: 0.8380, std: 0.0132.


# Model

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

import time

In [56]:
def train_model(X, X_test, y, params, folds, model_type='lgb', plot_feature_importance=False, 
                averaging='usual', model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if model_type == 'lgb':
            train_data = lgb.Dataset(X_train, label=y_train, params={'verbose': -1})
            valid_data = lgb.Dataset(X_valid, label=y_valid, params={'verbose': -1})
            
#             model = lgb.train(params,
#                     train_data,
#                     num_boost_round=20000,
#                     valid_sets = [train_data, valid_data],
#                     verbose_eval=False,
#                     early_stopping_rounds = 200)

            model = lgb.train(params,
                    train_data,
                    valid_sets = [train_data, valid_data],
                    verbose_eval=False)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values  
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = [f'col{ncol}' for ncol in range(X.shape[-1])]
            fold_importance["importance"] = model.feature_importance()
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                   by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction, scores
    
    else:
        return oof, prediction, scores

In [17]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

In [43]:
params = {'boost': 'gbdt',
          'feature_fraction': 0.3,
          'learning_rate': 0.01,
          'max_depth': -1,  
          'metric':'auc',
          'min_data_in_leaf': 31,
          'num_leaves': 151,
          'num_threads': -1,
          'verbosity': -1,
          'objective': 'binary',
         }

In [44]:
# CV mean score: 0.8404, std: 0.0101.

oof_lgb, prediction_lgb, scores = train_model(X, X_test_np, y, params=params, 
                                              folds=folds, model_type='lgb', 
                                              plot_feature_importance=False)

Fold 0 started at Sat Aug 15 13:56:30 2020
Fold 1 started at Sat Aug 15 13:56:36 2020
Fold 2 started at Sat Aug 15 13:56:41 2020
Fold 3 started at Sat Aug 15 13:56:45 2020
Fold 4 started at Sat Aug 15 13:56:50 2020
CV mean score: 0.8404, std: 0.0101.


In [63]:
params = {'boost': 'dart', 
          'n_estimators': 750,
          'feature_fraction': 0.3,
          'learning_rate': 0.01,
          'max_depth': -1,  
          'metric':'auc',
          'min_data_in_leaf': 15,
          'num_leaves': 151,
          'num_threads': -1,
          'verbosity': -1,
          'objective': 'binary'}

In [64]:
# CV mean score: 0.8391, std: 0.0113.

oof_lgb2, prediction_lgb2, scores = train_model(X, X_test_np, y, params=params, 
                                              folds=folds, model_type='lgb', 
                                              plot_feature_importance=False)

Fold 0 started at Sat Aug 15 14:08:00 2020


C:\Users\SakaevRF\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fold 1 started at Sat Aug 15 14:08:10 2020


C:\Users\SakaevRF\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fold 2 started at Sat Aug 15 14:08:21 2020


...

Fold 3 started at Sat Aug 15 14:08:33 2020


C:\Users\SakaevRF\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Fold 4 started at Sat Aug 15 14:08:44 2020


C:\Users\SakaevRF\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


CV mean score: 0.8391, std: 0.0113.


# Train other

In [65]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [66]:
def train_other_model(X, X_test, y, params, folds, model_type='svc', plot_feature_importance=False, 
                      averaging='usual', model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    
#     poly = PolynomialFeatures(2, interaction_only=True)
#     X = poly.fit_transform(X)
#     X_test = poly.transform(X_test)
    
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_valid = scaler.transform(X_valid)
        X_test_scaled = scaler.transform(X_test)
        
        if model_type == 'svc':
            model = SVC(**params)
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test_scaled)[:, 1]
        elif model_type == 'neighbors':
            model = KNeighborsClassifier(**params)
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test_scaled)[:, 1]
        elif model_type == 'lr':
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test_scaled)[:, 1]
        elif model_type == 'rf':
            model = RandomForestClassifier(**params)
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test_scaled)[:, 1]
        else:
            raise FileNotFoundError("INCORRECT MODEL TYPE")
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values  
        

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    return oof, prediction, scores

In [81]:
# CV mean score: 0.8218, std: 0.0111.

svc_params = {'C': 1.0, 'probability': True, 'gamma': 'scale'}

oof_svc, prediction_svc, scores = train_other_model(X, X_test_np, y, params=svc_params, 
                                                    folds=folds, model_type='svc')

# oof_lgb, prediction_lgb, scores = train_other_model(X_train_ica, X_test_ica, y, params=svc_params, 
#                                                     folds=folds, model_type='svc')

Fold 0 started at Sun Aug 16 21:15:25 2020
Fold 1 started at Sun Aug 16 21:15:39 2020
Fold 2 started at Sun Aug 16 21:15:53 2020
Fold 3 started at Sun Aug 16 21:16:07 2020
Fold 4 started at Sun Aug 16 21:16:21 2020
CV mean score: 0.8392, std: 0.0101.


In [89]:
# CV mean score: 0.8407, std: 0.0091.

neighbors_params = {'n_neighbors': 30, 'weights': 'distance', 'p': 1}

oof_kn, prediction_kn, scores = train_other_model(X, X_test_np, y, params=neighbors_params, 
                                                    folds=folds, model_type='neighbors')

# oof_lgb, prediction_lgb, scores = train_other_model(X_train_ica, X_test_ica, y, params=neighbors_params, 
#                                                     folds=folds, model_type='neighbors')

Fold 0 started at Sun Aug 16 21:20:13 2020
Fold 1 started at Sun Aug 16 21:20:14 2020
Fold 2 started at Sun Aug 16 21:20:16 2020
Fold 3 started at Sun Aug 16 21:20:18 2020
Fold 4 started at Sun Aug 16 21:20:19 2020
CV mean score: 0.8407, std: 0.0091.


In [97]:
# CV mean score: 0.6166, std: 0.0088.

lr_params = {'C': 0.001, 'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 5000}

oof_lr, prediction_lr, scores = train_other_model(X, X_test_np, y, params=lr_params, 
                                                    folds=folds, model_type='lr')

# oof_lgb, prediction_lgb, scores = train_other_model(X_train_ica, X_test_ica, y, params=svc_params, 
#                                                     folds=folds, model_type='svc')

Fold 0 started at Sun Aug 16 21:21:05 2020
Fold 1 started at Sun Aug 16 21:21:05 2020
Fold 2 started at Sun Aug 16 21:21:05 2020
Fold 3 started at Sun Aug 16 21:21:05 2020
Fold 4 started at Sun Aug 16 21:21:05 2020
CV mean score: 0.6189, std: 0.0093.


In [108]:
# CV mean score: 0.8347, std: 0.0126.

rf_params = {'n_estimators': 500, 'max_depth': None, 'n_jobs': -1, 'min_samples_split': 5}

oof_rf, prediction_rf, scores = train_other_model(X, X_test_np, y, params=rf_params, 
                                                    folds=folds, model_type='rf')

# oof_lgb, prediction_lgb, scores = train_other_model(X_train_ica, X_test_ica, y, params=svc_params, 
#                                                     folds=folds, model_type='svc')

Fold 0 started at Sun Aug 16 21:39:13 2020
Fold 1 started at Sun Aug 16 21:39:17 2020
Fold 2 started at Sun Aug 16 21:39:21 2020
Fold 3 started at Sun Aug 16 21:39:25 2020
Fold 4 started at Sun Aug 16 21:39:30 2020
CV mean score: 0.8383, std: 0.0121.


In [109]:
preds = np.concatenate([oof_lgb.reshape(-1, 1), oof_lgb2.reshape(-1, 1), 
                oof_svc.reshape(-1, 1), oof_kn.reshape(-1, 1), 
                oof_rf.reshape(-1, 1)], axis=1)
preds = pd.DataFrame(preds, columns=['lgb', 'lgb2', 'svc', 'kn', 'rf'])

In [110]:
test_preds = np.concatenate([prediction_lgb.reshape(-1, 1), prediction_lgb2.reshape(-1, 1), 
                prediction_svc.reshape(-1, 1), prediction_kn.reshape(-1, 1), 
                prediction_rf.reshape(-1, 1)], axis=1)
test_preds = pd.DataFrame(test_preds, columns=['lgb', 'lgb2', 'svc', 'kn', 'rf'])

In [111]:
preds.corr()

,lgb,lgb2,svc,kn,rf
lgb,1.000000,0.987882,0.953312,0.948892,0.972896
lgb2,0.987882,1.000000,0.956961,0.952089,0.979435
svc,0.953312,0.956961,1.000000,0.943292,0.952594
kn,0.948892,0.952089,0.943292,1.000000,0.943898
rf,0.972896,0.979435,0.952594,0.943898,1.000000


In [132]:
# 0.8436028884445137
new_pred = np.average(preds[['lgb', 'svc', 'kn', 'rf']], axis=1, weights=[1.3, 2, 3, 0.5])
roc_auc_score(y, new_pred)

0.8436028884445137

In [142]:
# 0.8437484635901519
new_pred = np.average(preds[['lgb', 'svc', 'kn']], axis=1, weights=[1, 1, 2])
roc_auc_score(y, new_pred)

0.8437484635901519

In [133]:
# 0.83692

from hyperopt import hp, fmin, tpe, space_eval

def to_max(hpp):
    new_pred = np.average(preds[['lgb2', 'svc', 'kn', 'rf', 'lgb']], axis=1, 
                          weights=[hpp['lgb2'], hpp['svc'], hpp['kn'], hpp['rf'], hpp['lgb']])
    return -roc_auc_score(y, new_pred)

aver_params = {'lgb2': hp.uniform('lgb2', 0, 4),
               'svc': hp.uniform('svc', 0, 4),
               'kn': hp.uniform('kn', 0, 4),
               'rf': hp.uniform('rf', 0, 4),
               'lgb': hp.uniform('lgb', 0, 4),
              }

best = fmin(to_max, aver_params, algo=tpe.suggest, max_evals=1000)

print(best)

100%|██████████████████████████████████████████████| 1000/1000 [00:18<00:00, 52.68it/s, best loss: -0.8437479835402076]
{'kn': 3.8568572141616038, 'lgb': 1.837302000554893, 'lgb2': 0.0014531829962610154, 'rf': 0.004864621896966322, 'svc': 1.8928043500484077}


In [143]:
weights=[1, 1, 2]

new_pred = np.average(preds[['lgb', 'svc', 'kn']], axis=1, 
                      weights=weights)
prediction_lgb = np.average(test_preds[['lgb', 'svc', 'kn']], axis=1, 
                            weights=weights)
roc_auc_score(y, new_pred)

0.8437484635901519

In [92]:
prediction_lgb = np.average(test_preds[['lgb2', 'svc', 'kn', 'rf', 'lgb']], axis=1, 
                            weights=[best['lgb2'], best['svc'], best['kn'], best['rf'], best['lgb']])

In [88]:
prediction_lgb = test_preds.mean(axis=1)

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

# rf = LogisticRegression(random_state=0, n_jobs=-1)
# distributions = dict(n_estimators=[50, 100, 250, 500], max_depth=[9, 11, 15, None])
# >>> clf = RandomizedSearchCV(logistic, distributions, random_state=0)
# >>> search = clf.fit(iris.data, iris.target)
# >>> search.best_params_

# Stacking

In [56]:
ens_model = RandomForestClassifier(n_estimators=1000)
ens_model.fit(preds, y)
prediction_lgb = ens_model.predict_proba(test_preds)[:, 1]

In [62]:
prediction_lgb

array([0.176, 0.278, 0.065, ..., 0.21 , 0.165, 0.915])

In [147]:
ens_model = LogisticRegression()
ens_model.fit(preds, y)
prediction_lgb = ens_model.predict_proba(test_preds)[:, 1]

C:\Users\SakaevRF\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# RFECV

In [13]:
from sklearn.feature_selection import RFECV

rf_params = {'n_estimators': 500, 'max_depth': None, 'n_jobs': -1, 'min_samples_split': 5}
estimator = RandomForestClassifier(**rf_params)

selector = RFECV(estimator, step=1, cv=5, scoring='roc_auc')
selector = selector.fit(X, y)
selector.ranking_

array([ 1,  5,  1,  3,  4,  1,  1,  1,  1,  1,  7,  1,  1,  1, 10,  1,  9,
        1, 11,  2,  6,  8,  1,  1])

In [22]:
rf_params = {'n_estimators': 500, 'max_depth': None, 'n_jobs': -1, 
             'min_samples_split': 5, 'random_state': 0}
for i in range(selector.ranking_.min(), selector.ranking_.max() + 1):
    print('-' * 30)
    print(f"MASK {i}")
    mask = selector.ranking_ <= i

    oof_rf, prediction_rf, scores = train_other_model(X[:, mask], X_test_np[:, mask], y, params=rf_params, 
                                                        folds=folds, model_type='rf')

------------------------------
MASK 1
Fold 0 started at Thu Aug 13 10:40:34 2020
Fold 1 started at Thu Aug 13 10:40:36 2020
Fold 2 started at Thu Aug 13 10:40:39 2020
Fold 3 started at Thu Aug 13 10:40:41 2020
Fold 4 started at Thu Aug 13 10:40:44 2020
CV mean score: 0.8323, std: 0.0103.
------------------------------
MASK 2
Fold 0 started at Thu Aug 13 10:40:46 2020
Fold 1 started at Thu Aug 13 10:40:49 2020
Fold 2 started at Thu Aug 13 10:40:51 2020
Fold 3 started at Thu Aug 13 10:40:54 2020
Fold 4 started at Thu Aug 13 10:40:56 2020
CV mean score: 0.8321, std: 0.0110.
------------------------------
MASK 3
Fold 0 started at Thu Aug 13 10:40:59 2020
Fold 1 started at Thu Aug 13 10:41:02 2020
Fold 2 started at Thu Aug 13 10:41:05 2020
Fold 3 started at Thu Aug 13 10:41:08 2020
Fold 4 started at Thu Aug 13 10:41:11 2020
CV mean score: 0.8320, std: 0.0115.
------------------------------
MASK 4
Fold 0 started at Thu Aug 13 10:41:14 2020
Fold 1 started at Thu Aug 13 10:41:17 2020
Fold 2 st

In [25]:
mask = selector.ranking_ <= 1
mask

array([ True, False,  True, False, False,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True, False,  True,
       False, False, False, False,  True,  True])

## v2

In [13]:
poly = PolynomialFeatures(2, interaction_only=False)
X_poly = poly.fit_transform(X)
X_test_poly = poly.transform(X_test_np)

In [14]:
from sklearn.feature_selection import RFECV

rf_params = {'n_estimators': 500, 'max_depth': None, 'n_jobs': -1, 'min_samples_split': 5}
estimator = RandomForestClassifier(**rf_params)

selector = RFECV(estimator, step=1, cv=5, scoring='roc_auc')
selector = selector.fit(X_poly, y)
selector.ranking_

array([7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 6, 1, 1, 1, 1, 1, 1, 1])

In [15]:
rf_params = {'n_estimators': 500, 'max_depth': None, 'n_jobs': -1, 
             'min_samples_split': 5, 'random_state': 0}
for i in range(selector.ranking_.min(), selector.ranking_.max() + 1):
    print('-' * 30)
    print(f"MASK {i}")
    mask_poly = selector.ranking_ <= i

    oof_rf, prediction_rf, scores = train_other_model(X_poly[:, mask_poly], X_test_poly[:, mask_poly], y, 
                                                      params=rf_params, folds=folds, model_type='rf')

------------------------------
MASK 1
Fold 0 started at Thu Aug 13 13:54:13 2020
Fold 1 started at Thu Aug 13 13:54:21 2020
Fold 2 started at Thu Aug 13 13:54:29 2020
Fold 3 started at Thu Aug 13 13:54:37 2020
Fold 4 started at Thu Aug 13 13:54:44 2020
CV mean score: 0.8316, std: 0.0103.
------------------------------
MASK 2
Fold 0 started at Thu Aug 13 13:54:52 2020
Fold 1 started at Thu Aug 13 13:55:00 2020
Fold 2 started at Thu Aug 13 13:55:07 2020
Fold 3 started at Thu Aug 13 13:55:15 2020
Fold 4 started at Thu Aug 13 13:55:23 2020
CV mean score: 0.8308, std: 0.0101.
------------------------------
MASK 3
Fold 0 started at Thu Aug 13 13:55:30 2020
Fold 1 started at Thu Aug 13 13:55:38 2020
Fold 2 started at Thu Aug 13 13:55:46 2020
Fold 3 started at Thu Aug 13 13:55:54 2020
Fold 4 started at Thu Aug 13 13:56:01 2020
CV mean score: 0.8315, std: 0.0101.
------------------------------
MASK 4
Fold 0 started at Thu Aug 13 13:56:09 2020
Fold 1 started at Thu Aug 13 13:56:17 2020
Fold 2 st

## v3

In [16]:
poly = PolynomialFeatures(2, interaction_only=True)
X_poly = poly.fit_transform(X)
X_test_poly = poly.transform(X_test_np)

In [17]:
from sklearn.feature_selection import RFECV

rf_params = {'n_estimators': 500, 'max_depth': None, 'n_jobs': -1, 'min_samples_split': 5}
estimator = RandomForestClassifier(**rf_params)

selector = RFECV(estimator, step=1, cv=5, scoring='roc_auc')
selector = selector.fit(X_poly, y)
selector.ranking_

array([4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1])

In [18]:
rf_params = {'n_estimators': 500, 'max_depth': None, 'n_jobs': -1, 
             'min_samples_split': 5, 'random_state': 0}
for i in range(selector.ranking_.min(), selector.ranking_.max() + 1):
    print('-' * 30)
    print(f"MASK {i}")
    mask_poly = selector.ranking_ <= i

    oof_rf, prediction_rf, scores = train_other_model(X_poly[:, mask_poly], X_test_poly[:, mask_poly], y, 
                                                      params=rf_params, folds=folds, model_type='rf')

------------------------------
MASK 1
Fold 0 started at Thu Aug 13 15:03:56 2020
Fold 1 started at Thu Aug 13 15:04:03 2020
Fold 2 started at Thu Aug 13 15:04:09 2020
Fold 3 started at Thu Aug 13 15:04:16 2020
Fold 4 started at Thu Aug 13 15:04:22 2020
CV mean score: 0.8310, std: 0.0101.
------------------------------
MASK 2
Fold 0 started at Thu Aug 13 15:04:29 2020
Fold 1 started at Thu Aug 13 15:04:35 2020
Fold 2 started at Thu Aug 13 15:04:42 2020
Fold 3 started at Thu Aug 13 15:04:48 2020
Fold 4 started at Thu Aug 13 15:04:55 2020
CV mean score: 0.8298, std: 0.0097.
------------------------------
MASK 3
Fold 0 started at Thu Aug 13 15:05:02 2020
Fold 1 started at Thu Aug 13 15:05:08 2020
Fold 2 started at Thu Aug 13 15:05:15 2020
Fold 3 started at Thu Aug 13 15:05:22 2020
Fold 4 started at Thu Aug 13 15:05:29 2020
CV mean score: 0.8303, std: 0.0107.
------------------------------
MASK 4
Fold 0 started at Thu Aug 13 15:05:35 2020
Fold 1 started at Thu Aug 13 15:05:42 2020
Fold 2 st

# Hyperopt

In [251]:
X = np.concatenate([X_train.values, X_train_ica], axis=1)
y = y_train.values.flatten()
X_test_np = np.concatenate([X_test.values, X_test_ica], axis=1)

In [ ]:
params = {'boost': 'gbdt',
#           'feature_fraction': 0.5,
          'learning_rate': 0.01,
          'max_depth': -1,  
          'metric':'auc',
          'min_data_in_leaf': 21,
          'num_leaves': 251,
          'num_threads': -1,
          'verbosity': -1,
          'objective': 'binary',
         }

In [260]:
def to_max(hp_params):
    oof_lgb, prediction_lgb, scores = train_model(X, X_test_np, y, params=hp_params, 
                                              folds=folds, model_type='lgb', 
                                              plot_feature_importance=False)
    return -np.mean(scores)

lgb_params = {'boost': 'gbdt',
          'feature_fraction': hp.choice('feature_fraction',    np.arange(0.05, 1.01, 0.05)),
          'learning_rate': 0.01,
          'max_depth': hp.choice('max_depth', [-1, 7, 11, 15, 21]),  
          'metric': 'auc',
          'min_data_in_leaf': hp.choice('min_data_in_leaf',    np.arange(11, 151, 5)),
          'num_leaves': hp.choice('num_leaves',    np.arange(51, 351, 10)),
          'num_threads': -1,
          'verbosity': -1,
          'lambda_l2': hp.choice('lambda_l2',    np.arange(0, 3, 0.1)),
          'objective': 'binary',
         }

In [261]:
# minimize the objective over the space
from hyperopt import fmin, tpe, space_eval
best = fmin(to_max, lgb_params, algo=tpe.suggest, max_evals=100)

print(best)
print(space_eval(lgb_params, best))

Fold                                                                                                                   
0                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:16:06 2020                                                                                               
Fold                                                                                                                   
1                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:16:15 2020                                                                                               
Fold                                    

started at                                                                                                             
Mon Aug  3 10:19:05 2020                                                                                               
Fold                                                                                                                   
3                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:19:08 2020                                                                                               
Fold                                                                                                                   
4                                                                                                                      
started at                              

CV mean score: 0.8164, std: 0.0119.                                                                                    
Fold                                                                                                                   
0                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:21:48 2020                                                                                               
Fold                                                                                                                   
1                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:21:58 2020                

2                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:24:53 2020                                                                                               
Fold                                                                                                                   
3                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:24:57 2020                                                                                               
Fold                                                                                                                   
4                                       

Mon Aug  3 10:28:54 2020                                                                                               
CV mean score: 0.8327, std: 0.0120.                                                                                    
Fold                                                                                                                   
0                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:29:00 2020                                                                                               
Fold                                                                                                                   
1                                                                                                                      
started at                              

...

1                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:56:22 2020                                                                                               
Fold                                                                                                                   
2                                                                                                                      
started at                                                                                                             
Mon Aug  3 10:56:30 2020                                                                                               
Fold                                                                                                                   
3                                       

Mon Aug  3 11:00:50 2020                                                                                               
Fold                                                                                                                   
4                                                                                                                      
started at                                                                                                             
Mon Aug  3 11:00:54 2020                                                                                               
CV mean score: 0.8353, std: 0.0120.                                                                                    
Fold                                                                                                                   
0                                                                                                                      
started at                              

Fold                                                                                                                   
1                                                                                                                      
started at                                                                                                             
Mon Aug  3 11:04:39 2020                                                                                               
Fold                                                                                                                   
2                                                                                                                      
started at                                                                                                             
Mon Aug  3 11:04:47 2020                                                                                               
Fold                                    

started at                                                                                                             
Mon Aug  3 11:09:34 2020                                                                                               
Fold                                                                                                                   
4                                                                                                                      
started at                                                                                                             
Mon Aug  3 11:09:42 2020                                                                                               
CV mean score: 0.8353, std: 0.0121.                                                                                    
Fold                                                                                                                   
0                                       

Mon Aug  3 11:14:14 2020                                                                                               
Fold                                                                                                                   
1                                                                                                                      
started at                                                                                                             
Mon Aug  3 11:14:17 2020                                                                                               
Fold                                                                                                                   
2                                                                                                                      
started at                                                                                                             
Mon Aug  3 11:14:20 2020                

In [272]:
print(space_eval(lgb_params, best))

{'boost': 'gbdt', 'feature_fraction': 0.7500000000000001, 'lambda_l2': 0.6000000000000001, 'learning_rate': 0.01, 'max_depth': 15, 'metric': 'auc', 'min_data_in_leaf': 31, 'num_leaves': 131, 'num_threads': -1, 'objective': 'binary', 'verbosity': -1}


In [ ]:
np.arange(0.05, 1.01, 0.05)[14]

# Saving

In [144]:
prediction_lgb

array([0.10710884, 0.3671106 , 0.16721723, ..., 0.15495443, 0.21209393,
       0.80144842])

In [145]:
subm.iloc[:, 0] = prediction_lgb

In [146]:
subm.to_csv('input/subm021.csv', header=None, index=None)